In [54]:
import pandas as pd
import os

# Cleaning Data

## Cleaning + Eliminating Columns

In [55]:
df = pd.read_csv(r"C:\Users\alexa\Documents\0 - FILES\ACADEMICS\BRANDEIS UNIVERSITY\2023-2024\FALL 2023\COSI 101a Fundamentals of A.I\MBTA_Reliability.csv")
df = df[df['mode_type'].str.contains('bus', case=False)]
df = df[df['metric_type'].str.contains('Headway / Schedule Adherence', case=False)]

In [56]:
df['service_date'] = pd.to_datetime(df['service_date']).dt.date
df

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId
7,2023-09-30,553,553,NaN,Express Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,39.0,55.0,NaN,8
8,2023-09-30,34E,34E,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,448.0,837.0,NaN,9
9,2023-09-30,36,36,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,342.0,441.0,NaN,10
10,2023-09-30,411,411,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,58.0,153.0,NaN,11
11,2023-09-30,429,429,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,122.0,192.0,NaN,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
817289,2016-01-01,104,104,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,143.0,244.0,0.0,817290
817290,2016-01-01,101,101,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,227.0,260.0,0.0,817291
817291,2016-01-01,100,100,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,195.0,206.0,0.0,817292
817292,2016-01-01,10,10,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,256.0,335.0,0.0,817293


In [57]:
df['reliability_percentage'] = (df['otp_numerator'] / df['otp_denominator']) * 100
df['unreliable_percentage'] = 100 - df['reliability_percentage']

In [58]:
columns_to_remove = ['gtfs_route_short_name',
                     'gtfs_route_long_name', 
                     'gtfs_route_desc',
                     'route_category',
                     'metric_type',
                     'cancelled_numerator', 
                     'otp_numerator', 
                     'otp_denominator',
                     'ObjectId']
df = df.drop(columns=columns_to_remove, axis=1)
df.head(10)

,service_date,gtfs_route_id,mode_type,peak_offpeak_ind,reliability_percentage,unreliable_percentage
7,2023-09-30,553,Bus,OFF_PEAK,70.909091,29.090909
8,2023-09-30,34E,Bus,OFF_PEAK,53.524492,46.475508
9,2023-09-30,36,Bus,OFF_PEAK,77.551020,22.448980
10,2023-09-30,411,Bus,OFF_PEAK,37.908497,62.091503
11,2023-09-30,429,Bus,OFF_PEAK,63.541667,36.458333
12,2023-09-30,43,Bus,OFF_PEAK,57.407407,42.592593
13,2023-09-30,44,Bus,OFF_PEAK,60.137457,39.862543
14,2023-09-30,441,Bus,OFF_PEAK,51.724138,48.275862
15,2023-09-30,450,Bus,OFF_PEAK,46.006390,53.993610
16,2023-09-30,455,Bus,OFF_PEAK,50.998185,49.001815


## Adding Weather Data

In [59]:
df_weather = pd.read_csv(r"C:\Users\alexa\Documents\0 - FILES\ACADEMICS\BRANDEIS UNIVERSITY\2023-2024\FALL 2023\COSI 101a Fundamentals of A.I\Final Project\101a-Artificial-Intelligence-Project\data\boston_weather.csv")
df_weather['time'] = pd.to_datetime(df_weather['time'], format='%Y-%m-%d')

df_weather.head()

,time,tavg,prcp
0,2013-03-01,3.2,0.0
1,2013-03-02,3.1,0.0
2,2013-03-03,2.6,0.0
3,2013-03-04,1.8,0.0
4,2013-03-05,2.9,0.0


In [60]:
df['service_date'] = pd.to_datetime(df['service_date'])
df_weather['time'] = pd.to_datetime(df_weather['time'])

# Set 'time' as index in df_weather for easier mapping
df_weather.set_index('time', inplace=True)

# Map values from df_weather to df based on 'service_date'
df['average_temp'] = df['service_date'].map(df_weather['tavg'])
df['precipitation'] = df['service_date'].map(df_weather['prcp'])

In [61]:
df = df[df['service_date'] <= '2023-03-01']
df = df.dropna()
df

,service_date,gtfs_route_id,mode_type,peak_offpeak_ind,reliability_percentage,unreliable_percentage,average_temp,precipitation
57074,2023-03-01,99,Bus,OFF_PEAK,76.111111,23.888889,2.4,0.0
57076,2023-03-01,99,Bus,PEAK,71.666667,28.333333,2.4,0.0
57078,2023-03-01,97,Bus,OFF_PEAK,76.404494,23.595506,2.4,0.0
57080,2023-03-01,97,Bus,PEAK,61.428571,38.571429,2.4,0.0
57082,2023-03-01,96,Bus,PEAK,63.829787,36.170213,2.4,0.0
...,...,...,...,...,...,...,...,...
817289,2016-01-01,104,Bus,OFF_PEAK,58.606557,41.393443,3.9,0.0
817290,2016-01-01,101,Bus,OFF_PEAK,87.307692,12.692308,3.9,0.0
817291,2016-01-01,100,Bus,OFF_PEAK,94.660194,5.339806,3.9,0.0
817292,2016-01-01,10,Bus,OFF_PEAK,76.417910,23.582090,3.9,0.0


## Creating for Bus 553

In [62]:
# df_553 = df[df['gtfs_route_id'] == '553']
# df_553

In [63]:
output_file = r"C:\Users\alexa\Documents\0 - FILES\ACADEMICS\BRANDEIS UNIVERSITY\2023-2024\FALL 2023\COSI 101a Fundamentals of A.I\Final Project\101a-Artificial-Intelligence-Project\data\MBTA_Bus_Reliability.csv"

df.to_csv(output_file, index=False)